<a href="https://colab.research.google.com/github/Nichanan670510741/Nichanan/blob/main/229351_LAB12_670510741.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ปฏิบัติการครั้งที่ 10 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

1. ให้เริ่มทำปฏิบัติการจาก colab notebook/ipynb ที่กำหนดให้ จากนั้น share แล้วส่ง link ใน Mango

ดาวน์โหลดข้อมูลผู้รับวินิจฉัยโรคหัวใจใน link ข้างล่างนี้  
https://donlapark.pages.dev/229351/data/framingham.csv

In [2]:
# uploading the csv file to colab

!wget -O framingham.csv https://donlapark.pages.dev/229351/data/framingham.csv

--2025-09-28 17:18:20--  https://donlapark.pages.dev/229351/data/framingham.csv
Resolving donlapark.pages.dev (donlapark.pages.dev)... 172.66.47.56, 172.66.44.200, 2606:4700:310c::ac42:2f38, ...
Connecting to donlapark.pages.dev (donlapark.pages.dev)|172.66.47.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191803 (187K) [text/csv]
Saving to: ‘framingham.csv’

framingham.csv      100%[===================>] 187.31K  --.-KB/s    in 0.02s   

2025-09-28 17:18:21 (7.97 MB/s) - ‘framingham.csv’ saved [191803/191803]



In [3]:
# import module ที่ต้องใช้
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('framingham.csv')

#drop rows with missing values
data = data.dropna()

print(data.info())

data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 3658 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             3658 non-null   int64  
 1   age              3658 non-null   int64  
 2   education        3658 non-null   float64
 3   currentSmoker    3658 non-null   int64  
 4   cigsPerDay       3658 non-null   float64
 5   BPMeds           3658 non-null   float64
 6   prevalentStroke  3658 non-null   int64  
 7   prevalentHyp     3658 non-null   int64  
 8   diabetes         3658 non-null   int64  
 9   totChol          3658 non-null   float64
 10  sysBP            3658 non-null   float64
 11  diaBP            3658 non-null   float64
 12  BMI              3658 non-null   float64
 13  heartRate        3658 non-null   float64
 14  glucose          3658 non-null   float64
 15  TenYearCHD       3658 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 485.8 KB
None


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


### ตัวแปรตามคือ `TenYearCHD` มีค่าเป็น 1 เมื่อผู้มาใช้บริการมีโรคหัวใจภายใน 10 ปีหลังจากการตรวจสุขภาพ

### แบ่งเป็นตัวแปรต้น `X` กับตัวแปรตาม `y`

In [5]:
y = data['TenYearCHD']

X = data.drop(['TenYearCHD'], axis = 1)

### สร้าง Logistic regression

$$ p(y=1|x_1,\ldots,x_p) = \frac{1}{1+e^{-(a_0+a_1x_1+\ldots+a_px_p)}} $$

### วิธีที่ 1: ใช้ `scikit-learn` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html))

In [6]:
from sklearn.linear_model import LogisticRegression

# สร้าง model object ก่อน
# ตั้ง random state ให้เป็น 0 เพื่อให้ผลลัพธ์ของทุกคนเหมือนกัน
model = LogisticRegression()

# แล้วค่อยใส่ข้อมูลโดยใช้ `fit`
model.fit(X,y)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Warning บอกว่ามีปํญหาในการ optimize เนื่องจากบางค่าใน `X` มีขนาดใหญ่เกินไป (ตัวอย่างเช่นตัวแปร `totChol` มีค่า `>200`)

### เป็นตัวอย่างหนึ่งที่แสดงให้เห็นถึงความสำคัญของ standardization (ปรับให้ mean ของแต่ละตัวแปรเป็น 0 และ s.d. เป็น 1)  

### สมมติให้ `X1` เป็นตัวแปรหนึ่งใน dataset  

### Standardization ทำโดยการแปลง `X1 -> (X1 - mean(X1))/(s.d.(X1))`

In [7]:
X.std()

,0
male,0.496886
age,8.562029
education,1.022656
currentSmoker,0.499949
cigsPerDay,11.921590
BPMeds,0.171557
prevalentStroke,0.075561
prevalentHyp,0.463229
diabetes,0.162292
totChol,44.097681


In [8]:
# Standardization
X_s = (X-X.mean())/X.std()

X_s

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,1.119602,-1.232411,1.974939,-0.978230,-0.757065,-0.176877,-0.075976,-0.672768,-0.166761,-0.948978,-1.193947,-1.078733,0.292010,0.356321,-0.203016
1,-0.892931,-0.414848,0.019247,-0.978230,-0.757065,-0.176877,-0.075976,-0.672768,-0.166761,0.298253,-0.514811,-0.160096,0.724911,1.608249,-0.244850
2,1.119602,-0.181259,-0.958599,1.021975,0.920563,-0.176877,-0.075976,-0.672768,-0.166761,0.184868,-0.220518,-0.243609,-0.108914,-0.060988,-0.495852
3,-0.892931,1.337073,0.997093,1.021975,1.759377,-0.176877,-0.075976,1.485991,-0.166761,-0.268670,0.798187,1.009079,0.688016,-0.895606,0.884661
4,-0.892931,-0.414848,0.997093,1.021975,1.172207,-0.176877,-0.075976,-0.672768,-0.166761,1.091946,-0.107329,0.090441,-0.659878,0.773630,0.131654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1.119602,0.052331,-0.958599,1.021975,-0.673184,-0.176877,-0.075976,1.485991,-0.166761,1.726900,2.111184,0.758541,0.046044,-0.812144,0.173488
4234,1.119602,0.169126,0.997093,1.021975,2.849836,-0.176877,-0.075976,-0.672768,-0.166761,-0.676855,-0.265794,-0.243609,-1.493704,-0.895606,-0.579519
4237,-0.892931,0.285920,0.019247,-0.978230,-0.757065,-0.176877,-0.075976,-0.672768,-0.166761,0.729115,0.051136,0.006929,-1.060803,0.356321,1.051996
4238,1.119602,-1.115616,0.997093,-0.978230,-0.757065,-0.176877,-0.075976,1.485991,-0.166761,-1.175747,0.390705,1.259616,-0.044963,-0.728682,-0.412184


### ใช้ `X_s` แทน `X` แล้ว warning จะหายไป

In [9]:
# warning จะหายไป
model.fit(X_s,y)

LogisticRegression()

In [10]:
# ดู intercept (a_0) ที่ได้โดยใช้
print('intercept:',model.intercept_)

# ดู coefficient ที่เหลือทั้งหมด (a_1,a_2,...,a_p) โดยใช้
print('coefficients:',model.coef_)

intercept: [-1.99225983]
coefficients: [[ 0.27433427  0.54274307 -0.04913154  0.03548216  0.21359737  0.02799904
   0.05215207  0.10871821  0.00597986  0.10254482  0.33767256 -0.04762226
   0.02702066 -0.03913318  0.17089461]]


### ดูผลจากการใช้โมเดลแบ่งกลุ่ม `X_s` โดยใช้ `model.predict(X_s)`

In [11]:
y_pred = model.predict(X_s)

print(y_pred)

[0 0 0 ... 0 0 0]


### โมเดลแบ่งให้ข้อมูลส่วนใหญ่อยู่ในกลุ่มที่ 0 ทำให้เราไม่มั่นใจว่ามีข้อมูลที่ถูกแบ่งให้อยู่ในกลุ่มที่ 1 หรือไม่

### สามารถนับจำนวนข้อมูลที่ถูกแบ่งให้อยู่ในกลุ่มที่ 1 โดยคำนวณผลบวกของตัวเลขทั้งหมดที่อยู่ใน `y_pred`

In [12]:
# คำนวณผลบวกของตัวเลขทั้งหมดที่อยู่ใน y_pred

print(y_pred.sum())

70


In [13]:
y

,TenYearCHD
0,0
1,0
2,0
3,1
4,0
...,...
4233,1
4234,0
4237,0
4238,0


### เราสามารถสร้าง array ที่ตรวจสอบว่าค่าที่ model ทำนาย (`y_pred`) ตรงกับค่าที่แท้จริง (`y`) หรือไม่โดยใช้ `y==y_pred`

| y | y_pred |       | y == y_pred |
|:------:|:------:|:-----:|:----------------:|
|    1   |    1   | $\to$ |         1        |
|    0   |    0   | $\to$ |         1        |
|    0   |    1   | $\to$ |         0        |
|    1   |    0   | $\to$ |         0        |

In [14]:
# จะเห็นว่าโมเดลแบ่งกลุ่มให้ข้อมูลชิ้นที่ 3 ไม่ตรงกับกลุ่มที่แท้จริง

y == y_pred

,TenYearCHD
0,True
1,True
2,True
3,False
4,True
...,...
4233,False
4234,True
4237,True
4238,True


In [15]:
True + False

1

### จากตรงนี้ เราสามารถนับจำนวนข้อมูลที่โมเดลแบ่งกลุ่มได้ถูกต้องโดยใช้ `(y == y_pred).sum()` (เพราะว่าในการบวกค่า boolean นั้น `True`=0 และ `False`=1)

In [16]:
(y == y_pred).sum()

np.int64(3133)

## Exercise 1:

### ให้ `y_true` เป็นค่าที่แท้จริงทั้งหมด และ `y_pred` เป็นค่าที่โมเดลทำนายทั้งหมด ค่า accuracy คำนวณได้จาก

$$\text{Accuracy} = \frac{\# (\texttt{y_true} = \texttt{y_pred})}{N}$$

### โดยที่ $N$ คือจำนวนข้อมูลทั้งหมด

### จงสร้างฟังก์ชันที่คำนวณค่า Accuracy โดยมี inputs เป็น `y_true` และ `y_pred`

### Hint: ใช้ `y_true.shape[0]` ในการนับจำนวนข้อมูลทั้งหมด

In [17]:
def accuracy(y_true, y_pred):

  ########################
  #Fill in your code here#
  ########################
    N = y_true.shape[0]
    correct = np.sum(y_true == y_pred)

    accuracy = correct / N
    return accuracy

# ทดสอบฟังก์ชัน accuracy
print(accuracy(np.array([1,0,0,0]),np.array([0,1,1,1]))) #ควรจะได้ 0
print(accuracy(np.array([1,0,0,0]),np.array([1,0,0,0]))) #ควรจะได้ 1
print(accuracy(np.array([1,1,0,0]),np.array([1,0,1,1]))) #ควรจะได้ 0.25

0.0
1.0
0.25


## Exercise 2:

จงคำนวณ accuracy ของโมเดล logistic regression ที่ได้ข้างบน

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris

# ฟังก์ชัน accuracy ของเราเอง
def accuracy(y_true, y_pred):
    N = y_true.shape[0]
    correct = np.sum(y_true == y_pred)
    return correct / N

# โหลด dataset
iris = load_iris()
X = iris.data
y = iris.target

# แบ่งข้อมูล train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# สร้างและ train logistic regression
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# ทำนาย
y_pred = model.predict(X_test)

# คำนวณ accuracy ด้วยฟังก์ชันของเรา
acc = accuracy(y_test, y_pred)
print("Accuracy =", acc)


Accuracy = 1.0


### วิธีที่ 2: ใช้ `Logit` จาก `statsmodels` ([Documentation](https://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.Logit.html))

### ดูค่า Z-value และ p-value ของแต่ละตัวแปร

In [19]:
from statsmodels.discrete.discrete_model import Logit
from sklearn.datasets import load_iris
import statsmodels.api as sm


iris = load_iris()
X = iris.data
y = (iris.target == 0).astype(int)   # binary: setosa (1) vs not setosa (0)

# สร้าง DataFrame
X_df = pd.DataFrame(X, columns=iris.feature_names)

# ใส่ constant (intercept)
X_s = sm.add_constant(X_df)
# โมเดลชื่อ logit
logit = Logit(y, X_s)

# จะเห็นว่า syntax ของ statsmodels นั้นใส่ข้อมูลก่อนแล้วค่อย fit
logit_fitted = logit.fit()

print(logit_fitted.summary())

         Current function value: 0.000000
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  150
Model:                          Logit   Df Residuals:                      145
Method:                           MLE   Df Model:                            4
Date:                Sun, 28 Sep 2025   Pseudo R-squ.:                   1.000
Time:                        17:18:25   Log-Likelihood:            -6.9724e-09
converged:                      False   LL-Null:                       -95.477
Covariance Type:            nonrobust   LLR p-value:                 3.305e-40
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               -11.8568   4.64e+05  -2.55e-05      1.000    -9.1e+05     9.1e+05
sepal length (cm)     9.6294   1.21e+05   7.

/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.12/dist-packa

## Exercise 3:

### จงระบุว่ามีตัวแปรไหนบ้างที่มีความสำคัญที่ระดับนัยสำคัญ 0.1

ตัวแปรที่ p-value < 0.1 ได้แก่:
sepal length (p ≈ 0.051)
sepal width (p ≈ 0.029)
petal length (p ≈ 0.000)
petal width (p ≈ 0.069)

### ผลลัพธ์ที่ได้จากการใช้ `predict` จะแตกต่างจาก `scikit-learn` ตรงที่ output เป็น**ค่าความน่าจะเป็น** ไม่ใช่ค่าการแบ่งกลุ่ม

In [20]:
logit_fitted.predict(X_s)

,0
0,1.000000e+00
1,1.000000e+00
2,1.000000e+00
3,1.000000e+00
4,1.000000e+00
...,...
145,3.694326e-28
146,2.766648e-27
147,4.774253e-27
148,1.444486e-30


In [21]:
import numpy as np
import statsmodels.api as sm
from sklearn.datasets import load_iris
from statsmodels.discrete.discrete_model import Logit

# โหลดข้อมูล Iris
iris = load_iris()
X = iris.data
y = (iris.target == 0).astype(int)  # binary: setosa vs not setosa

# ใส่ constant
X_s = sm.add_constant(X)

# Fit model
logit = Logit(y, X_s)
logit_fitted = logit.fit()

# predict probability
y_prob = logit_fitted.predict(X_s)

print(y_prob[:5])  # แสดง 5 ตัวแรก


         Current function value: 0.000000
         Iterations: 35
[1. 1. 1. 1. 1.]


/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.12/dist-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.12/dist-packa

## Exercise 4:

ฟังก์ชันข้างล่างนี้มีหน้าที่เปลี่ยนค่าความน่าจะเป็นที่ได้ให้เป็นการแบ่งกลุ่ม (0 หรือ 1) จงเติมส่วนที่หายไปในฟังก์ชันข้างล่างนี้ให้สมบูรณ์

In [22]:
def classify(y_pred):
  '''
  y_pred: numpy array of probability predictions
  Return: a list of classifications (0 or 1)
  '''
  N = y_pred.shape[0] #number of observations
  results = [] #array of classifications

  for prob in y_pred:
    if prob >= 0.5 :
      results.append(1)
    else:
      results.append(0)

  return results


#Test
print(classify(np.array([0.2,0.4,0.8]))) #should return [0,0,1]
print(classify(np.array([0.8,0.6,0.1]))) #should return [1,1,0]

[0, 0, 1]
[1, 1, 0]


## Exercise 5:

จงแสดงการแบ่งกลุ่ม (classification) ที่ได้จากโมเดล `logit_fitted` ในพื้นที่ข้างล่างนี้


In [28]:
type(X)

numpy.ndarray

In [31]:
import statsmodels.api as sm

X_with_const = sm.add_constant(X)

y_prob = logit_fitted.predict(X_with_const)
y_class = (y_prob >= 0.5).astype(int)

print("classification:")
print(y_class[:20])

# เทียบกับค่าจริง
import pandas as pd
df_result = pd.DataFrame({
    "y_true": y,
    "y_pred": y_class
})
df_result.head(20)


classification:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


,y_true,y_pred
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1
